# Amazon Bedrock AgentCore Runtime でシンプルな CrewAI エージェントを Amazon Bedrock モデルでホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime を使用してシンプルな CrewAI エージェントをホスティングする方法を学びます。OpenTelemetry インストルメンテーションと AWS OpenTelemetry Python ライブラリを使用してこのエージェントに可観測性を追加し、Amazon CloudWatch GenAI Observability ダッシュボードでパフォーマンスを監視します。

### チュートリアル詳細

| 情報               | 詳細                                                                             |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 対話型                                                                           |
| エージェントタイプ   | シングル                                                                         |
| エージェントフレームワーク | CrewAI                                                                     |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                     |
| チュートリアル構成   | AgentCore Runtime でのエージェントホスティング。CrewAI と Amazon Bedrock モデルの使用 |
| チュートリアル分野   | クロスバーティカル                                                               |
| 難易度              | 簡単                                                                             |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK と boto3                                     |

### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* Amazon Bedrock モデルの使用
* CrewAI の使用
* Amazon CloudWatch GenAI Observability


### チュートリアルアーキテクチャ

このチュートリアルでは、既存のマルチエージェント Crew を AgentCore Runtime にデプロイする方法について説明します。

デモンストレーション目的で、Amazon Bedrock モデルを使用した CrewAI Crew を使用します。

この例では、Web 検索機能を持つトラベルエージェントを使用します。
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

## 前提条件

このチュートリアルを実行するには以下が必要です:
* Python 3.10+
* 適切な権限を持つ AWS 認証情報
* Amazon Bedrock AgentCore SDK
* CrewAI
* Amazon CloudWatch アクセス
* Amazon CloudWatch で [Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) を有効化

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## AgentCore Runtime へのデプロイ準備

次に、エージェントを AgentCore Runtime にデプロイしましょう。そのためには以下が必要です:
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
* `app = BedrockAgentCoreApp()` でコードに App を初期化
* `@app.entrypoint` デコレータで呼び出し関数をデコレート
* `app.run()` で AgentCoreRuntime にエージェントの実行を制御させる

### Amazon Bedrock モデルを使用した CrewAI エージェント
Amazon Bedrock モデルを使用して、Runtime 対応の CrewAI エージェントを作成しましょう。

In [ ]:
%%writefile crewai_runtime_agent.py
import os

from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from ddgs import DDGS
import logging
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from opentelemetry.instrumentation.crewai import CrewAIInstrumentor

# CrewAI を OpenTelemetry でインストルメント
# 注意: AWS OpenTelemetry distro は opentelemetry-instrument コマンド使用時に
# トレーサープロバイダーのセットアップを自動的に処理します
CrewAIInstrumentor().instrument()

app = BedrockAgentCoreApp()

# ロギングの設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@tool("web_search")
def web_search(query: str) -> str:
    """旅行先、観光スポット、イベントに関する最新情報を Web で検索します。"""
    try:
        ddgs = DDGS()
        results = ddgs.text(query, max_results=3)
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result.get('title', 'タイトルなし')}\n"
                f"   {result.get('body', '概要なし')}\n"
                f"   ソース: {result.get('href', 'URL なし')}\n"
            )
        
        return "\n".join(formatted_results) if formatted_results else "結果が見つかりませんでした。"
        
    except Exception as e:
        return f"検索エラー: {str(e)}"

def get_llm():
    model_id = os.getenv("BEDROCK_MODEL_ID", "global.anthropic.claude-haiku-4-5-20251001-v1:0")
    region = os.getenv("AWS_DEFAULT_REGION", "us-west-2")
    
    try:
        llm = LLM(
            model=f"bedrock/{model_id}",
            temperature=0.7,
            max_tokens=512,
            aws_region_name=region
        )
        logger.info(f"Bedrock LLM を正常に初期化しました。モデル: {model_id}、リージョン: {region}")
        return llm
    except Exception as e:
        logger.error(f"Bedrock LLM の初期化に失敗しました: {str(e)}")
        logger.error("適切な AWS 認証情報が設定されていることと、Bedrock モデルへのアクセス権があることを確認してください")
        raise

@app.entrypoint
def crewai_agent_bedrock(payload, context):
    """
    ペイロードでエージェントを呼び出します
    """
    print(f'ペイロード: {payload}')
    try:
        user_input = payload.get("prompt", "おすすめの観光スポットは何ですか？")
        print(f"リクエスト処理中: {user_input}")
        
        llm = get_llm()

        travel_agent = Agent(
            role='Travel Destination Researcher',
            goal='Find dream destinations matching user preferences using web search for current information',
            backstory="あなたはリアルタイムの Web 情報にアクセスできる、パーソナライズされた旅行推奨を専門とする経験豊富なトラベルエージェントです。",
            verbose=True,
            allow_delegation=False,
            llm=llm,
            max_iter=3,
            tools=[web_search]
        )

        task = Task(
            description=f"Research and provide travel recommendations based on this request: {user_input}. Use web search to find current information about venues, events, and attractions.",
            expected_output="A comprehensive list of recommended destinations with current information, brief descriptions, and practical travel details.",
            agent=travel_agent
        )

        crew = Crew(
            agents=[travel_agent],
            tasks=[task],
            verbose=True
        )

        result = crew.kickoff()
        
        print("コンテキスト:\n-------\n", context)
        print("生の結果:\n*******\n", result.raw)
        
        return {"result": result.raw}
        
    except Exception as e:
        print(f'例外が発生しました: {e}')
        return {"error": f"エラーが発生しました: {str(e)}"}

if __name__ == "__main__":
    app.run()

## 内部で何が起こるか？

`BedrockAgentCoreApp` を使用すると、自動的に以下が行われます:

* ポート 8080 でリッスンする HTTP サーバーの作成
* エージェントの要件を処理するための `/invocations` エンドポイントの実装
* ヘルスチェック用の `/ping` エンドポイントの実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプとレスポンス形式の処理
* AWS 標準に従ったエラー処理の管理

## AgentCore Runtime へのエージェントのデプロイ

`CreateAgentRuntime` オペレーションは、コンテナイメージ、環境変数、暗号化設定を指定できる包括的な設定オプションをサポートしています。また、プロトコル設定（HTTP、MCP）や認可メカニズムを設定して、クライアントがエージェントと通信する方法を制御することもできます。

**注意:** 運用のベストプラクティスは、CI/CD パイプラインと IaC を使用してコードをコンテナとしてパッケージ化し、ECR にプッシュすることです。

このチュートリアルでは、Amazon Bedrock AgentCore Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイします。

### AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して、エントリポイント、作成した実行ロール、および requirements ファイルで AgentCore Runtime デプロイを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Dockerfile が生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

`bedrock_agentcore_starter_toolkit` を使用してエージェントを設定する場合、OpenTelemetry インストルメンテーションを自動的に処理することに注意してください。

コンテナ化環境（Docker など）を設定する場合は、以下のコマンドを追加してください。例を示します:

`CMD ["opentelemetry-instrument", "python", "runtime_agent_main.py"]`

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "simple_crewai_travel_agent"
response = agentcore_runtime.configure(
    entrypoint="crewai_runtime_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    memory_mode='NO_MEMORY'
)
response

### AgentCore Runtime へのエージェントの起動

Dockerfile ができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
# Disable CrewAI's built-in telemetry to avoid conflicts
launch_result = agentcore_runtime.launch(env_vars={
        "CREWAI_DISABLE_TELEMETRY": "true",
        "OTEL_PYTHON_EXCLUDED_URLS": "https://api.scarf.sh/"
    })
launch_result

### AgentCore Runtime ステータスの確認
AgentCore Runtime をデプロイしたので、デプロイステータスを確認しましょう。

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime の呼び出し

最後に、ペイロードで AgentCore Runtime を呼び出すことができます。

<div style="text-align:left">
    <img src="images/invoke.png" width=85%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "What are some cowboy-themed attractions and museums in Texas?"})
invoke_response

### 呼び出し結果の処理

呼び出し結果を処理して、アプリケーションに組み込むことができます。

In [ ]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### boto3 を使用した AgentCore Runtime の呼び出し

AgentCore Runtime が作成されたら、任意の AWS SDK で呼び出すことができます。例えば、boto3 の `invoke_agent_runtime` メソッドを使用できます。

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What are some rodeo events happening in Oklahoma?"})
)

response_body = boto3_response['response'].read()
response_data = json.loads(response_body)
display(Markdown(response_data.get('result', 'No result found')))

### Amazon CloudWatch での AgentCore Observability

まとめると、AgentCore Runtime でホストされたエージェントから可観測性を有効にするには、以下の手順に従ってください:

- Amazon CloudWatch で Transaction Search を有効化
- エージェントはトレースを発行し、OpenTelemetry コマンドを使用してインストルメント化されます: `opentelemetry-instrument python any_runtime_agent.py`
- requirements.txt ファイルには、Bedrock AgentCore Runtime にエージェントをデプロイする際に `aws-opentelemetry-distro` がリストされています。

## GenAI Observability ダッシュボードでの Bedrock AgentCore 概要

可観測性が有効になっているすべてのエージェントを表示し、時間枠に基づいてデータをフィルタリングできます。

メインダッシュボードでは、すべてのエージェントにわたる Runtime メトリクスを表示できます。

デプロイしたエージェントをクリックすると、このエージェント固有の Runtime メトリクスのダッシュボードに移動します。カスタム時間枠でデータをフィルタリングすることもできます。

Sessions View タブでは、このエージェントに関連するすべてのセッションに移動できます。

Trace View タブでは、Runtime 上のこのエージェントのトレースとスパン情報を確認できます。


<div style="text-align:left">
    <img src="images/span_crew_Ai.png" width="60%"/>
</div>


GenAI Observability ダッシュボードの様々な機能をクリックして、トレースに関するより詳細な情報を取得してください。

<div style="text-align:left">
    <img src="images/span_details.png" width="60%"/>
</div>

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう。

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！

可観測性を有効にしたシンプルな CrewAI エージェントを Amazon Bedrock AgentCore Runtime に正常に作成してデプロイしました。この例では以下を示しました:

- Web 検索機能を持つシンプルな CrewAI トラベルエージェントの作成
- Amazon CloudWatch を通じた可観測性の有効化
- SDK と boto3 の両方を使用したエージェントの呼び出し

エージェントは完全な可観測性と監視機能を備えて使用できるようになりました。